In [1]:
import cl_graph_bert as cgm
import torch
from torch import nn
import json
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
import tqdm

from torch.utils.data import Dataset, DataLoader

import dgl
g = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

In [2]:
model = cgm.CLIPGraphModel(
    rel_types = g.etypes,
    emb_types = {x: g.number_of_nodes(x) for x in g.ntypes} 
)

device = "cuda"

model.load_state_dict(torch.load("./base_statedict_5651.253525578349.pt", map_location=torch.device('cuda')))
model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CLIPGraphModel(
  (graph_model): RGCN(
    (embed): ParameterDict(
        (Brand): Parameter containing: [torch.FloatTensor of size 1900x512]
        (Customer): Parameter containing: [torch.FloatTensor of size 11041x512]
        (Product): Parameter containing: [torch.FloatTensor of size 5334x512]
        (Review): Parameter containing: [torch.FloatTensor of size 77060x512]
    )
    (conv1): HeteroGraphConv(
      (mods): ModuleDict(
        (rev_SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
        (SOLD_BY): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (REVIEW_OF): GraphConv(in=512, out=256, normalization=both, activation=None)
        (rev_WROTE): GraphConv(in=512, out=256, normalization=both, activation=None)
      )
    )
    (conv2): HeteroGraphConv(
     

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import tqdm

bert_base = BertModel.from_pretrained('bert-base-uncased')
device = "cuda"

bert_base.to(device)
model.language_model.to(device)
model.graph_model.to(device)
g.to(device)

reivew_ids = []

gnn_sims = []
bert_base_sims = []
bert_finetuned_sims = []

with torch.no_grad():

    for i in tqdm.tqdm(range(g.num_nodes('Product'))):
        review_id = g.successors(i, 'rev_REVIEW_OF').to(device)
        
        if len(review_id) > 1 and len(review_id) < 650:

            reivew_ids.append(review_id)
            
            gnn_embs = model.graph_model.embed["Review"][review_id].detach().cpu().numpy()
            
            base_bert_embs = bert_base(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()
            
            bert_finetuned_embs = model.language_model(input_ids = g.nodes["Review"].data['input_ids'][review_id].to(device), 
               attention_mask=g.nodes["Review"].data['attention_mask'][review_id].to(device), 
               token_type_ids=g.nodes["Review"].data['token_type_ids'][review_id].to(device)).last_hidden_state[:,0].type(torch.float64).detach().cpu().numpy()

            gnn_sims.append( (np.sum(cosine_similarity(gnn_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_base_sims.append( (np.sum(cosine_similarity(base_bert_embs)) - len(review_id)) /  (len(review_id) ** 2 - len(review_id)) )
            bert_finetuned_sims.append( (np.sum(cosine_similarity(bert_finetuned_embs)) - len(review_id)) / (len(review_id) ** 2 - len(review_id)) )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 71%|███████████████████████████████████████████████████████                      | 3812/5334 [10:33<02:35,  9.78it/s]

In [7]:
for i in range(len(gnn_sims)):
    print(gnn_sims[i], bert_base_sims[i], bert_finetuned_sims[i])

0.6277103913136017 0.8268821558012124 0.9572585000001964
0.4055873325892857 0.7837415820510998 0.9379011466384055
0.44336927259290543 0.7653988364540306 0.9448764257094822
0.34763701756795246 0.8021195736030341 0.9258848206244171
0.527640438079834 0.8181333275989203 0.9534615987984492
0.2689549446105957 0.8423244155791278 0.9237513259299732
0.44739793141682943 0.7435472558954691 0.9274969119772969
0.33617091178894043 0.7791316478116391 0.9146735984597884
0.4990874517531622 0.7784917828060489 0.9197278174715043
0.5146522521972656 0.7968453673194399 0.9178160285076664
0.4550503624810113 0.7949908159003145 0.9323048150353697
0.3370717046645702 0.7728560316364009 0.9313825661216691
0.30480160251740485 0.7944981703043286 0.9334168334406688
0.5551620682859733 0.7671140535778513 0.9271120603333256
0.44972767163348454 0.8116810987891644 0.93740780817199
0.38380499945746527 0.7687704494871236 0.9279873207991136
0.32195087507659315 0.7891434623546072 0.9316664433233618
0.4991954167683919 0.76415

0.5425691604614258 0.8207996748324898 0.9505564945387387
0.5069071451822916 0.7748557549918414 0.9443098409806365
0.34452160921963776 0.7803660545595027 0.9285072646079795
0.5811049492804559 0.7366378174140266 0.9381748059585763
0.35811227910658894 0.7410940208005601 0.9242729368603544
0.6722181774321057 0.7654753772147436 0.9469802464664321
0.6074076152983165 0.754443244518695 0.950912917831171
0.5738141840154475 0.8112920483052051 0.9358275689703768
0.7080381694592928 0.8398257699878203 0.9701981673726212
0.4924523489815848 0.7731675275630988 0.9390442526791772
0.6305161158243815 0.7840763049785747 0.9258226246903614
0.2091536521911621 0.833306262894707 0.9476580110556299
0.37946156734178044 0.7878208808707435 0.9338437125575446
0.5026321411132812 0.7551095313051335 0.9467332479216656
0.7654074136312906 0.8432201845290429 0.9637278298823487
0.6168196784125434 0.7838004572858608 0.9372778830305474
0.5153234481811524 0.802913240019749 0.9197273464037687
0.2435675960847701 0.79739497598

0.04598880536628492 0.8149708156955102 0.9293255514048646
0.023679060094496784 0.7710470951213709 0.9375405168775736
-0.038507336661929174 0.7861176284131304 0.9163514961691103
0.012970082171551593 0.7547222164373178 0.9482025222865372
0.018894720077514648 0.8000568593915464 0.9007481706799713
-0.0011252244313557944 0.747178471398067 0.9179810107449445
-0.03269262313842773 0.7756731285682871 0.92240844007087
0.1384739535195487 0.8036112771222942 0.9358812150321318
0.11174941062927246 0.8398491837753888 0.9418375772436887
0.05154509771437872 0.8394406034895684 0.9413963911200701
0.04122955458504813 0.7952394904542273 0.9224544249049494
-0.0002141426771115034 0.7819106170113446 0.9206895579138987
0.03474127451578776 0.7237983982460716 0.9132002546036517
-0.022295532804546936 0.7889104417171777 0.9667594724061228
0.007278745824640448 0.747357942278095 0.9225745924257696
0.0037121389104031966 0.8104375068240308 0.9320756496397965
-0.0007966700054350353 0.8045655741410093 0.9423143820524993

In [8]:
print("Average bert base", np.mean(bert_base_sims))
print("Average bert fine-tuned", np.mean(bert_finetuned_sims))
print("Average difference", np.mean(np.array(bert_finetuned_sims) - np.array(bert_base_sims)))


Average bert base 0.8007960083136799
Average bert fine-tuned 0.9359616202462319
Average difference 0.1351656119325517


In [10]:
print(np.mean(gnn_sims[:len(gnn_sims)//2]))
print(np.mean(gnn_sims[len(gnn_sims)//2:]))

0.5007986689840875
0.02319790494159576
